## Introduction

In this project I will use Ye

## Import Necessary libraries

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import pandas as pd
import numpy as np

Using TensorFlow backend.


### Get  data

In [24]:
df = pd.read_csv('reduced_review.csv', sep = '|', names = ['stars', 'text'], error_bad_lines=False)

In [25]:
df= df.dropna()
df = df[df.stars.apply(lambda x: x.isnumeric())]
df = df[df.stars.apply(lambda x: x !="")]

In [26]:
df = df[df.text.apply(lambda x: x !="")]

In [27]:
df.describe()

,stars,text
count,1673870,1673870
unique,5,1673452
top,5,Good stuff
freq,709732,6


In [33]:
df.head()

,stars,text
0,5,The minute I realized that Conflict was a bloc...
2,5,I love Conflict Kitchen. The food is fantasti...
3,4,Holy moly! I'm addicted!\n\nI first heard of C...
4,4,"Had some great Persian food, though it was mor..."
5,4,Yummy food. Good prices. Encourages me to try ...


In [48]:
labels = df['stars'].map(lambda x : 1 if int(x) > 3 else 0)

In [28]:
t = Tokenizer()
t.fit_on_texts(df['text'])
encoded_docs = t.texts_to_sequences(df['text'])

In [34]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=50)

In [36]:
print(data[0])

[    0     0     1   747     3  1513    13 12870     6     4  2348   250
   359    12   246    65   132     6     3  1050   320   401   196   126
    48   120    74   157    37    21   654     7 12870   574    55    42
     4   217   268    64  1086    11   355    37     7     1    28  3580
     3  1503]


## Build LSTM model

In [51]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=50))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(data, np.array(labels), validation_split=0.5, epochs=3, verbose=1)
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)

Train on 836935 samples, validate on 836935 samples
Epoch 1/3
352960/836935 [===========>..................] - ETA: 6972s - loss: 0.3179 - acc: 0.8632

In [ ]:
print('Accuracy: %f' % (accuracy*100))

## Build model Convolution layer

### Word2Vec

In [ ]:
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True, norm_only=True)
# getting word vectors of a word
dog = model['dog']
#performing king queen magic
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

#picking odd one out
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

#printing similarity index
print(model.similarity('woman', 'man'))

In [ ]:
sentence=[[‘Neeraj’,’Boy’],[‘Sarwan’,’is’],[‘good’,’boy’]]
model = gensim.models.Word2Vec(sentence, min_count=1,size=300,workers=4)
print(model.similarity('woman', 'man'))